In [30]:
import os
import tensorflow as tf

In [31]:
from keras.models import load_model

In [32]:
loaded_model_2 = load_model(os.path.join('saved_models','vgg16','my-vgg16.h5'))

In [33]:
loaded_model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 14847301 (56.64 MB)
Trainable params: 14847301 (56.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR='datasets/flowers/daisy'
FLOWER_SUNFLOWER_DIR='datasets/flowers/sunflower'
FLOWER_TULIP_DIR='datasets/flowers/tulip'
FLOWER_DANDI_DIR='datasets/flowers/dandelion'
FLOWER_ROSE_DIR='datasets/flowers/rose'

In [15]:
import cv2                  
import numpy as np  
from tqdm import tqdm

In [16]:
def assign_label(img,flower_type):
    return flower_type

In [17]:
def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

In [18]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(X))

make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(X))

make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))

make_train_data('Dandelion',FLOWER_DANDI_DIR)
print(len(X))

make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(X))

100%|████████████████████████████████████████████████████████████████████████████████| 764/764 [00:11<00:00, 65.59it/s]


764


100%|████████████████████████████████████████████████████████████████████████████████| 733/733 [00:12<00:00, 57.47it/s]


1497


100%|████████████████████████████████████████████████████████████████████████████████| 984/984 [00:15<00:00, 64.52it/s]


2481


100%|██████████████████████████████████████████████████████████████████████████████| 1052/1052 [00:15<00:00, 67.08it/s]


3533


100%|████████████████████████████████████████████████████████████████████████████████| 784/784 [00:12<00:00, 62.14it/s]

4317


In [19]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [20]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255
Y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [22]:
from sklearn.model_selection import train_test_split
import random as rn

In [23]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [25]:
from keras.preprocessing.image import ImageDataGenerator

In [26]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [27]:
batch_size=128

In [29]:
History = loaded_model_2.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = 1, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

C:\Users\alexs\AppData\Local\Temp\ipykernel_19784\4218994610.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  History = loaded_model_2.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),


25/25 [==============================] - 1582s 63s/step - loss: 0.4467 - accuracy: 0.8372 - val_loss: 0.5335 - val_accuracy: 0.8065
